In [1]:
! pip install -q kaggle
from google.colab import files

In [2]:
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"gaurangdangayach","key":"6258e0a532bce00c861188066164a474"}'}

In [3]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/

In [4]:
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets list

ref                                                           title                                              size  lastUpdated          downloadCount  
------------------------------------------------------------  ------------------------------------------------  -----  -------------------  -------------  
gpreda/reddit-vaccine-myths                                   Reddit Vaccine Myths                              229KB  2021-06-19 20:05:51           7866  
crowww/a-large-scale-fish-dataset                             A Large Scale Fish Dataset                          3GB  2021-04-28 17:03:01           4789  
imsparsh/musicnet-dataset                                     MusicNet Dataset                                   22GB  2021-02-18 14:12:19           1615  
dhruvildave/wikibooks-dataset                                 Wikibooks Dataset                                   1GB  2021-06-09 15:58:24           2261  
mathurinache/twitter-edge-nodes                               Tw

In [5]:
! kaggle competitions download -c tabular-playground-series-jun-2021

  0% 0.00/264k [00:00<?, ?B/s]
100% 264k/264k [00:00<00:00, 39.1MB/s]
  0% 0.00/3.14M [00:00<?, ?B/s]
100% 3.14M/3.14M [00:00<00:00, 94.4MB/s]
 77% 5.00M/6.49M [00:00<00:00, 52.1MB/s]
100% 6.49M/6.49M [00:00<00:00, 58.2MB/s]


In [7]:
! mkdir train
! unzip train.csv.zip -d train

mkdir: cannot create directory ‘train’: File exists
Archive:  train.csv.zip
replace train/train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [8]:
import pandas as pd
data=pd.read_csv("train/train.csv")

In [9]:
data.head()

,id,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_35,feature_36,feature_37,feature_38,feature_39,feature_40,feature_41,feature_42,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49,feature_50,feature_51,feature_52,feature_53,feature_54,feature_55,feature_56,feature_57,feature_58,feature_59,feature_60,feature_61,feature_62,feature_63,feature_64,feature_65,feature_66,feature_67,feature_68,feature_69,feature_70,feature_71,feature_72,feature_73,feature_74,target
0,0,0,0,6,1,0,0,0,0,7,0,0,0,3,0,1,0,0,3,3,1,0,2,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,11,0,0,0,0,0,9,0,0,0,0,0,0,3,0,1,3,0,0,0,0,0,0,0,1,1,0,0,3,0,0,0,0,0,0,2,0,0,Class_6
1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,1,0,Class_6
2,2,0,0,0,0,0,1,0,3,0,0,1,0,0,0,0,0,0,0,0,5,4,0,0,0,0,0,0,0,0,1,0,0,0,0,2,0,0,5,0,5,0,1,0,0,0,1,0,0,1,0,7,0,1,1,0,3,4,0,0,1,3,0,2,0,0,8,0,0,0,0,1,0,0,0,0,Class_2
3,3,0,0,7,0,1,5,2,2,0,1,2,0,5,0,0,4,0,0,22,2,1,0,0,0,0,3,0,37,0,1,3,13,0,10,0,3,1,1,0,7,0,0,2,0,1,0,0,0,0,0,0,10,0,0,25,1,0,1,2,0,2,0,7,0,0,0,0,4,0,2,2,0,4,3,0,Class_8
4,4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,2,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Class_2


In [10]:
import numpy as np

In [11]:
len(data)

200000

In [12]:
y=np.zeros([len(data),9])
y.shape

(200000, 9)

In [13]:
classes=data.iloc[:,-1].unique()
classes
classes.sort()
classes

array(['Class_1', 'Class_2', 'Class_3', 'Class_4', 'Class_5', 'Class_6',
       'Class_7', 'Class_8', 'Class_9'], dtype=object)

In [14]:
for i in range(len(data)):
  for j in range(9):
    if data.iloc[i,-1]==classes[j]:
      y[i,j]=1
    else:
      y[i,j]=0
y[0:5]

array([[0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0.]])

In [15]:
X=np.array(data.iloc[:,1:-1])
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [16]:
data_x=np.hstack([np.ones([len(data),1]),X])

In [17]:
wts=np.zeros([76,9])

In [18]:
def sigmoid(x):
    prob= 1/(1+np.exp(-x))
    return prob

In [19]:
def optimize(xt,y,wts,lr):
    m=len(xt)
    pred=sigmoid(np.dot(xt,wts))
    cost= (-1/m)*(np.sum((y*np.log(pred)) + ((1-y)*(np.log(1-pred)))))
    wts=wts-lr*(1/m)*np.dot(xt.T,pred-y)
    #print(cost)
    #print(wts.shape,y.shape,pred.shape,xt.shape)
    return wts,cost

In [20]:
lr=0.001
for i in range(100):
  xt=data_x[i*2000:(i+1)*2000]
  yt=y[i*2000:(i+1)*2000]
  totalcost=0
  for x in range(9):
    ytc=yt[:,x]
    wts[:,x],cost=optimize(xt,ytc,wts[:,x],lr)
    totalcost=totalcost+cost
  if i%10==0:
    print(totalcost/9)
print(wts.shape)

0.6931471805599454
0.6914856394721984
0.6898043789172988
0.6881603719994043
0.686553381857572
0.6849405404629323
0.6833062400320989
0.6817857405441201
0.6801312463442538
0.6785119177155712
(76, 9)


In [ ]:
wts

In [22]:
pred=sigmoid(np.dot(data_x,wts))
category=np.zeros([len(data),1])
print(pred.shape)
for i in range(len(data)):
  for j in range(9):
    if pred[i,j]==max(pred[i,:]):
      category[i]=j
      break
print(pred[0:5])
print(category[0:5])
print(y[0:5])
#print(np.hstack([ans,y]))
#accuracy= np.where(ans == y,1, 0)
#accuracy_percent=sum(accuracy)/len(y)*100
#accuracy_percent

(200000, 9)
[[0.48841619 0.49045731 0.489157   0.48800301 0.48780453 0.50103988
  0.48823076 0.49070901 0.48978452]
 [0.48872509 0.49973584 0.49316957 0.48822565 0.48770277 0.49513361
  0.48644591 0.48278447 0.49171295]
 [0.4888066  0.49486957 0.49124576 0.48822006 0.48788594 0.49361342
  0.48819874 0.48941141 0.49135438]
 [0.48873057 0.47967309 0.48486637 0.48806035 0.48840452 0.49385611
  0.49275964 0.50720046 0.49000033]
 [0.48881951 0.49978407 0.49328413 0.48825937 0.48774519 0.49350827
  0.48668519 0.48359769 0.49194972]]
[[5.]
 [1.]
 [1.]
 [7.]
 [1.]]
[[0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0.]]


In [23]:
for i in range(len(data)):
  pred[i]=pred[i]/np.sum(pred[i])

In [24]:
! mkdir test
! unzip test.csv.zip -d train

Archive:  test.csv.zip
  inflating: train/test.csv          


In [25]:
test_data=pd.read_csv("train/test.csv")

In [26]:
X_test=np.array(test_data.iloc[:,1:])
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_test= scaler.fit_transform(X_test)

In [27]:
data_x_test=np.hstack([np.ones([len(test_data),1]),X_test])

In [28]:
pred_test=sigmoid(np.dot(data_x_test,wts))
category_test=np.zeros([len(test_data),1])

In [29]:
for i in range(len(test_data)):
  pred_test[i]=pred_test[i]/np.sum(pred_test[i])

In [30]:
columns = ['Class_1','Class_2','Class_3','Class_4','Class_5','Class_6','Class_7','Class_8','Class_9']

df= pd.DataFrame(pred_test,
                     columns=columns)

In [31]:
df.head()

,Class_1,Class_2,Class_3,Class_4,Class_5,Class_6,Class_7,Class_8,Class_9
0,0.110742,0.113422,0.111836,0.110624,0.110497,0.111949,0.110198,0.109271,0.111462
1,0.110736,0.112301,0.111369,0.110616,0.110531,0.111953,0.110547,0.110612,0.111334
2,0.110722,0.112068,0.111252,0.110603,0.110525,0.112511,0.110518,0.110559,0.111242
3,0.110719,0.111707,0.111141,0.110640,0.110543,0.111941,0.110721,0.111306,0.111282
4,0.110694,0.111788,0.111153,0.110596,0.110534,0.112218,0.110627,0.111129,0.111261


In [32]:
df1=pd.concat([test_data.iloc[:,0],df], axis=1)
df1.head()

,id,Class_1,Class_2,Class_3,Class_4,Class_5,Class_6,Class_7,Class_8,Class_9
0,200000,0.110742,0.113422,0.111836,0.110624,0.110497,0.111949,0.110198,0.109271,0.111462
1,200001,0.110736,0.112301,0.111369,0.110616,0.110531,0.111953,0.110547,0.110612,0.111334
2,200002,0.110722,0.112068,0.111252,0.110603,0.110525,0.112511,0.110518,0.110559,0.111242
3,200003,0.110719,0.111707,0.111141,0.110640,0.110543,0.111941,0.110721,0.111306,0.111282
4,200004,0.110694,0.111788,0.111153,0.110596,0.110534,0.112218,0.110627,0.111129,0.111261


In [ ]:
df1.to_csv("predictions1.csv")

In [ ]:
files.download('predictions1.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>